<a href="https://colab.research.google.com/github/oBrunoz/df-no-mapa/blob/main/land_surface_temperature.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import ee
import geemap.core as geemap

# Autenticar o Earth Engine no Colab
ee.Authenticate()
ee.Initialize(project="ee-alencarbdev")


In [37]:
# Applies scaling factors
def apply_scale_factors(image):
    optical_bands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
    thermal_bands = image.select('ST_B.*').multiply(0.00341802).add(149.0)
    return image.addBands(optical_bands, None, True).addBands(thermal_bands, None, True)

# Function to Mask Clouds
def cloud_mask(image):
    cloud_shadow_bitmask = (1 << 3)
    cloud_bitmask = (1 << 5)
    qa = image.select('QA_PIXEL')
    mask = qa.bitwiseAnd(cloud_shadow_bitmask).eq(0).And(
            qa.bitwiseAnd(cloud_bitmask).eq(0))
    return image.updateMask(mask)

# Carrega os limites administrativos
ra_df = ee.FeatureCollection('projects/ee-alencarbdev/assets/RA_DF_2025').filter(ee.Filter.eq('ra_codigo', 'RA-IX'))

# Processa a imagem
image = (ee.ImageCollection("LANDSAT/LC09/C02/T1_L2")
          .filterBounds(ra_df)
          .filterDate('2023-01-01', '2023-12-31')
          .map(apply_scale_factors)
          .map(cloud_mask)
          .median()
          .clip(ra_df))

ndvi = image.normalizedDifference(['SR_B5', 'SR_B4']).rename('NDVI')

def calculate_min_and_max_ndvi():
    stats = ndvi.reduceRegion(
        reducer=ee.Reducer.minMax(),
        geometry=ra_df.geometry(),
        scale=30,
        maxPixels=1e9
    ).getInfo()

    ndviMin = stats['NDVI_min']
    ndviMax = stats['NDVI_max']

    print("Minimum NDVI:", ndviMin);
    print("Maximum NDVI:", ndviMax);

    return ndviMin, ndviMax

def ndvi_layer(image):
    ndviPalette = {
    'min': -1,
    'max': 1,
    'palette': ['blue', 'white', 'green']
    };
    return {'ndvi': ndvi, 'ndviPallete': ndviPalette}

def thermal_layer(ndvi_min, ndvi_max):
    fraction_of_vegetation = ((ndvi.subtract(ndvi_min)).divide(ndvi_max - ndvi_min)).pow(2).rename('FV')

    land_surface_emissivity = fraction_of_vegetation.multiply(ee.Number(0.004)).add(ee.Number(0.986)).rename('EM')

    thermal = image.select('ST_B10').rename('thermal')

    land_surface_temperature = thermal.expression('(TB / (1 + (0.00115 * (TB / 1.438)) * log(em))) - 273.15', {'TB': thermal.select('thermal'), 'em': land_surface_emissivity}
                                                  ).rename(f'Temperatura do solo do Distrito Federal - 2025')

    return land_surface_temperature


def get_landsat_image(start_date, end_date):
    # Parâmetros de visualização (true color: bandas 4, 3, 2)
    visualization = {
        'bands': ['SR_B4', 'SR_B3', 'SR_B2'],
        'min': 0.0,
        'max': 0.15
    }

    # Obtém os parâmetros para visualização no frontend
    map_info = image.getMapId(visualization)

    return {
        'mapid': map_info['mapid'],
        'image': image,
        'token': map_info['token'],
        'visualization': visualization,
        # 'bounds': ra_df.geometry().bounds().getInfo()['coordinates'],
        'region_name': "Distrito Federal",
        'dates': {'start': start_date, 'end': end_date}
    }

landsat_value = get_landsat_image('2023-01-01', '2023-12-31')
ndvi_value = ndvi_layer(landsat_value['image'])
ndvi_min, ndvi_max = calculate_min_and_max_ndvi()
thermal_value = thermal_layer(ndvi_min, ndvi_max)
ground_cover = ee.FeatureCollection('projects/ee-alencarbdev/assets/COBERTURA_SOLO_DF_2019')

map = geemap.Map(center=[-15.8, -47.9], zoom=12)

# MAPA DO LANDSAT
map.add_layer(landsat_value['image'], landsat_value['visualization'], 'Landsat')

# MAPA DE NDVI
map.add_layer(ndvi_value['ndvi'], ndvi_value['ndviPallete'], 'NDVI')

# MAPA DE TEMPERATURA
map.add_layer(thermal_value, {
    "min": 18.47,
    "max": 42.86,
    "palette": [
      '040274', '040281', '0502a3', '0502b8', '0502ce', '0502e6',
      '0602ff', '235cb1', '307ef3', '269db1', '30c8e2', '32d3ef',
      '3be285', '3ff38f', '86e26f', '3ae237', 'b5e22e', 'd6e21f',
      'fff705', 'ffd611', 'ffb613', 'ff8b13', 'ff6e08', 'ff500d',
      'ff0000', 'de0101', 'c21301', 'a71001', '911003'
    ]
}, 'Temperatura do solo - ' + str(2025) + ' - Distrito Federal');

# MAPA DE COBERTURA DO SOLO
map.add_layer(ground_cover, {"color": "orange"}, "Cobertura do Solo")

map

Minimum NDVI: -0.9991615427613199
Maximum NDVI: 0.9080725767105766


EEException: Collection.loadTable: Collection asset 'projects/ee-alencarbdev/assets/COBERTURA_SOLO_DF_2019' not found.

In [36]:
import pandas as pd

def get_stats(image, band_name, region, scale=30):
    reducer = ee.Reducer.mean().combine(ee.Reducer.min(), '', True).combine(ee.Reducer.max(), '', True)
    stats = image.select(band_name).reduceRegion(
        reducer=reducer,
        geometry=region.geometry(),
        scale=scale,
        maxPixels=1e9
    )
    return stats.getInfo()

ceilandia = ra_df.filter(ee.Filter.eq('ra_codigo', 'RA-IX')).first()
# planaltina = ra_df.filter(ee.Filter.eq('ra_codigo', 'RA-VI'))

# Aplica as estatísticas para NDVI e Temperatura
ndvi_stats = get_stats(ndvi, 'NDVI', ceilandia)
thermal_stats = get_stats(thermal_value, thermal_value.bandNames().getInfo()[0], ceilandia)

# Monta o DataFrame
data = {
    'Região': ['Ceilândia'],
    'NDVI_Médio': [ndvi_stats.get('NDVI_mean')],
    'NDVI_Mínimo': [ndvi_stats.get('NDVI_min')],
    'NDVI_Máximo': [ndvi_stats.get('NDVI_max')],
    'Temp_Média': [thermal_stats.get('{}_mean'.format(thermal_value.bandNames().getInfo()[0]))],
    'Temp_Mínima': [thermal_stats.get('{}_min'.format(thermal_value.bandNames().getInfo()[0]))],
    'Temp_Máxima': [thermal_stats.get('{}_max'.format(thermal_value.bandNames().getInfo()[0]))],
}

df = pd.DataFrame(data)
print(df)

      Região  NDVI_Médio  NDVI_Mínimo  NDVI_Máximo  Temp_Média  Temp_Mínima  \
0  Ceilândia    0.479923    -0.999216     0.908073   32.618036    24.394269   

   Temp_Máxima  
0    41.019719  


# Organização em Classe

In [9]:
import ee
import geemap.core as geemap

class LandsatProcessor:
    def __init__(self, region_name="Distrito Federal", year=2023):
        ee.Authenticate()
        ee.Initialize(project="ee-alencarbdev")
        self.region_name = region_name
        self.year = year
        self.region = self._load_region()
        self.image = self._process_image()
        self.ndvi = self.image.normalizedDifference(['SR_B5', 'SR_B4']).rename('NDVI')
        self.ndvi_min, self.ndvi_max = self._calculate_min_and_max_ndvi()
        self.thermal = self._calculate_thermal_layer()

    def _load_region(self):
        return ee.FeatureCollection('FAO/GAUL/2015/level1') \
                 .filter(ee.Filter.eq('ADM1_NAME', self.region_name))

    def _apply_scale_factors(self, image):
        optical = image.select('SR_B.').multiply(0.0000275).add(-0.2)
        thermal = image.select('ST_B.*').multiply(0.00341802).add(149.0)
        return image.addBands(optical, None, True).addBands(thermal, None, True)

    def _cloud_mask(self, image):
        cloud_shadow = (1 << 3)
        cloud = (1 << 5)
        qa = image.select('QA_PIXEL')
        mask = qa.bitwiseAnd(cloud_shadow).eq(0).And(
               qa.bitwiseAnd(cloud).eq(0))
        return image.updateMask(mask)

    def _process_image(self):
        return (ee.ImageCollection("LANDSAT/LC09/C02/T1_L2")
                  .filterBounds(self.region)
                  .filterDate(f'{self.year}-01-01', f'{self.year}-12-31')
                  .map(self._apply_scale_factors)
                  .map(self._cloud_mask)
                  .median()
                  .clip(self.region))

    def _calculate_min_and_max_ndvi(self):
        stats = self.ndvi.reduceRegion(
            reducer=ee.Reducer.minMax(),
            geometry=self.region.geometry(),
            scale=30,
            maxPixels=1e9
        ).getInfo()

        print("Minimum NDVI:", stats['NDVI_min'])
        print("Maximum NDVI:", stats['NDVI_max'])
        return stats['NDVI_min'], stats['NDVI_max']

    def get_landsat_layer(self):
        vis = {
            'bands': ['SR_B4', 'SR_B3', 'SR_B2'],
            'min': 0.0,
            'max': 0.15
        }
        map_info = self.image.getMapId(vis)
        return {
            'mapid': map_info['mapid'],
            'image': self.image,
            'token': map_info['token'],
            'visualization': vis,
            'bounds': self.region.geometry().bounds().getInfo()['coordinates'],
            'region_name': self.region_name,
            'dates': {'start': f'{self.year}-01-01', 'end': f'{self.year}-12-31'}
        }

    def get_ndvi_layer(self):
        return {
            'ndvi': self.ndvi,
            'ndviPallete': {
                'min': -1,
                'max': 1,
                'palette': ['blue', 'white', 'green']
            }
        }

    def _calculate_thermal_layer(self):
        fv = ((self.ndvi.subtract(self.ndvi_min))
              .divide(self.ndvi_max - self.ndvi_min)).pow(2).rename('FV')
        em = fv.multiply(ee.Number(0.004)).add(ee.Number(0.986)).rename('EM')
        thermal = self.image.select('ST_B10').rename('thermal')
        lst = thermal.expression(
            '(TB / (1 + (0.00115 * (TB / 1.438)) * log(em))) - 273.15',
            {'TB': thermal, 'em': em}
        ).rename(f'Temperatura do solo do {self.region_name} - {self.year}')
        return lst

    def get_thermal_layer(self):
        return {
            'layer': self.thermal,
            'vis_params': {
                "min": 18.47,
                "max": 42.86,
                "palette": [
                    '040274', '040281', '0502a3', '0502b8', '0502ce', '0502e6',
                    '0602ff', '235cb1', '307ef3', '269db1', '30c8e2', '32d3ef',
                    '3be285', '3ff38f', '86e26f', '3ae237', 'b5e22e', 'd6e21f',
                    'fff705', 'ffd611', 'ffb613', 'ff8b13', 'ff6e08', 'ff500d',
                    'ff0000', 'de0101', 'c21301', 'a71001', '911003'
                ]
            },
            'name': f'Temperatura do solo - {self.year} - {self.region_name}'
        }

    def visualize(self):
        m = geemap.Map(center=[-15.8, -47.9], zoom=12)

        landsat = self.get_landsat_layer()
        ndvi = self.get_ndvi_layer()
        thermal = self.get_thermal_layer()

        m.add_layer(landsat['image'], landsat['visualization'], 'Landsat')
        m.add_layer(ndvi['ndvi'], ndvi['ndviPallete'], 'NDVI')
        m.add_layer(thermal['layer'], thermal['vis_params'], thermal['name'])

        return m


In [10]:
processor = LandsatProcessor(region_name="Distrito Federal", year=2025)
mapa = processor.visualize()
mapa

EEException: Image.normalizedDifference: No band named 'SR_B5'. Available band names: [].

# TESTES

In [ ]:
%pip install -q --upgrade altair
import altair as alt

In [ ]:
landsat_values = (ee.ImageCollection("LANDSAT/LC09/C02/T1_L2")
                  .filterDate('2023-01', '2023-02')
                  .first())

cities = ee.FeatureCollection([
    ee.Feature(ee.Geometry.Point(-15.8, -47.9), {'city': 'DF'}),
    ee.Feature(ee.Geometry.Point(-118.24, 34.05), {'city': 'Los Angeles'}),
    ee.Feature(ee.Geometry.Point(103.83, 1.33), {'city': 'Singapore'}),
])

regions = landsat_values.reduceRegions(cities, ee.Reducer.first())

city_climates_dataframe = ee.data.computeFeatures(
    {'expression': regions, 'fileFormat': 'PANDAS_DATAFRAME'}
)

alt.Chart(city_climates_dataframe).mark_bar(size=100).encode(
    alt.X('city:N', sort='y', axis=alt.Axis(labelAngle=0), title='City'),
    alt.Y('temperature_2m:Q', title='Temperature (K)'),
    tooltip=[
        alt.Tooltip('city:N', title='City'),
        alt.Tooltip('temperature_2m:Q', title='Temperature (K)'),
    ],
).properties(title='January 2023 temperature for selected cities', width=500)